In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

inputs_a = np_to_var(x_right[:160,0:1,:,None], dtype=np.float32)

inputs_b = np_to_var(x_rest[:160,0:1,:,None], dtype=np.float32)
inputs = [inputs_a, inputs_b]

In [ ]:
for i_class in range(2):
    ins = var_to_np(inputs[i_class].squeeze())
    bps = np.abs(np.fft.rfft(ins.squeeze()))
    angles = np.angle((np.fft.rfft(ins.squeeze())))
    for i_bp in range(bps.shape[1]):
        fig = plt.figure(figsize=(8,4))
        seaborn.distplot(bps[:,i_bp], color=seaborn.color_palette()[i_class], bins=80, rug=True)
        plt.title("Amplitudes")
        display(fig)
        plt.close(fig)
        fig = plt.figure(figsize=(8,4))
        seaborn.distplot(angles[:,i_bp] % (2*np.pi), color=seaborn.color_palette()[i_class], bins=80, rug=True)
        plt.title("Phase")
        display(fig)
        plt.close(fig)
        

In [ ]:
i_class = 1
i_bp = 18
ins = var_to_np(inputs[i_class].squeeze())
bps = np.abs(np.fft.rfft(ins.squeeze()))
angles = np.angle((np.fft.rfft(ins.squeeze())))
fig = plt.figure(figsize=(8,4))
seaborn.distplot(angles[:,i_bp] % (2*np.pi), color=seaborn.color_palette()[i_class], bins=80, rug=True)
plt.title("Phase")
display(fig)
plt.close(fig)

### match uniform

In [ ]:
from reversible.uniform import get_uniform_samples

### match with DSF/sigmoid logic

https://arxiv.org/abs/1804.00779

In [ ]:
i_bp = 18
points = np_to_var(angles[:,i_bp], dtype=np.float32)
sorted_points = th.sort(points, dim=0)[0].detach()

set_random_seeds(13123, False)

weights = th.randn(200, requires_grad=True)
bias = th.randn(200, requires_grad=True)
alphas = th.randn(200, requires_grad=True)


def inv_sigmoid(p, eps=1e-8):
    return th.log(p + eps) - th.log(1-p + eps)

samples = th.linspace(-3,3).unsqueeze(1)
out = inv_sigmoid(th.mean(th.sigmoid(alphas.unsqueeze(0)) * th.sigmoid(samples *
                                   th.exp(weights).unsqueeze(0) 
                                  + bias.unsqueeze(0)), dim=1))

# 0.43 without transformation
mean = th.zeros(1, requires_grad=True)
log_std = th.zeros(1, requires_grad=True)
optim = th.optim.Adam([mean,log_std, weights, bias, alphas], lr=1e-1)

n_epochs = 300
for i_epoch in range(n_epochs):
    std = th.exp(log_std)

    samples = get_uniform_samples(len(angles) * 8, mean, std * 2 * np.pi)
    transformed_samples = inv_sigmoid(th.mean(th.sigmoid(alphas.unsqueeze(0)) * th.sigmoid(samples *
                                   th.exp(weights).unsqueeze(0) 
                                  + bias.unsqueeze(0)), dim=1))
    sorted_samples, _ = th.sort(transformed_samples, dim=0)

    diffs = sorted_samples.reshape(len(points), -1) - sorted_points.unsqueeze(1)
    loss = th.sqrt(th.mean(diffs * diffs))
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Loss: {:.2f}".format(loss.item()))
        fig = plt.figure(figsize=(8,4))

        seaborn.distplot(var_to_np(sorted_points), color=seaborn.color_palette()[0], bins=80, rug=True)
        seaborn.distplot(var_to_np(sorted_samples), color=seaborn.color_palette()[1], bins=80, rug=True)
        display(fig)
        plt.close(fig)
        orig_sorted_samples = th.sort(samples, dim=0)[0]
        transformed_samples = inv_sigmoid(th.mean(th.sigmoid(alphas.unsqueeze(0)) * th.sigmoid(orig_sorted_samples *
                                   th.exp(weights).unsqueeze(0) 
                                  + bias.unsqueeze(0)), dim=1))
        fig = plt.figure(figsize=(8,4))
        plt.plot(var_to_np(orig_sorted_samples), var_to_np(transformed_samples))
        display(fig)
        plt.close(fig)

### leaky bounded relu

In [ ]:
i_bp = 18
points = np_to_var(angles[:,i_bp], dtype=np.float32)
sorted_points = th.sort(points, dim=0)[0].detach()

set_random_seeds(13123, False)
weights = th.randn(200, requires_grad=True)
bias = th.randn(200, requires_grad=True)
bounds_a = th.randn(200, requires_grad=True)
bounds_b = th.randn(200, requires_grad=True)
#weights.data *= 0.01
#bias.data *= 3


def leaky_bounded_relu(x,  lower_bounds, upper_bounds,leak_slope=0.01,):
    return x * leak_slope + th.min(th.max(x, lower_bounds.unsqueeze(0)), upper_bounds.unsqueeze(0))

In [ ]:
# 0.43 without transformation
mean = th.zeros(1, requires_grad=True)
log_std = th.zeros(1, requires_grad=True)
optim = th.optim.Adam([mean,log_std, weights, bias, bounds_a, bounds_b], lr=1e-1)

n_epochs = 300
for i_epoch in range(n_epochs):
    std = th.exp(log_std)
    samples = get_uniform_samples(len(angles) * 8, mean, std * 2 * np.pi)
    lower_bounds = th.min(bounds_a, bounds_b)
    upper_bounds = th.max(bounds_a, bounds_b)
    transformed_samples = th.mean(leaky_bounded_relu(
            samples * th.exp(weights).unsqueeze(0) + bias.unsqueeze(0),
            lower_bounds, upper_bounds), dim=1)
    sorted_samples, _ = th.sort(transformed_samples, dim=0)

    diffs = sorted_samples.reshape(len(points), -1) - sorted_points.unsqueeze(1)
    loss = th.sqrt(th.mean(diffs * diffs))
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Loss: {:.2f}".format(loss.item()))
        fig = plt.figure(figsize=(8,4))

        seaborn.distplot(var_to_np(sorted_points), color=seaborn.color_palette()[0], bins=80, rug=True)
        seaborn.distplot(var_to_np(sorted_samples), color=seaborn.color_palette()[1], bins=80, rug=True)
        display(fig)
        plt.close(fig)
        orig_sorted_samples = th.sort(samples, dim=0)[0]
        transformed_samples = th.mean(leaky_bounded_relu(
            orig_sorted_samples * th.exp(weights).unsqueeze(0) + bias.unsqueeze(0),
            lower_bounds, upper_bounds), dim=1)
        fig = plt.figure(figsize=(8,4))
        plt.plot(var_to_np(orig_sorted_samples), var_to_np(transformed_samples))
        display(fig)
        plt.close(fig)

## Inverse RevNet Logic

In [ ]:
from spectral_norm import spectral_norm

In [ ]:
i_bp = 18
points = np_to_var(angles[:,i_bp], dtype=np.float32)
sorted_points = th.sort(points, dim=0)[0].detach()
set_random_seeds(13123, False)
dist_net = nn.Sequential(
    spectral_norm(nn.Linear(1,200), to_norm=0.95),
    nn.ReLU(),
    spectral_norm(nn.Linear(200,1), to_norm=0.95))

mean = th.zeros(1, requires_grad=True)
log_std = th.zeros(1, requires_grad=True)
optim = th.optim.Adam([mean,log_std] + list(dist_net.parameters()), lr=1e-1)

n_epochs = 500
for i_epoch in range(n_epochs):
    std = th.exp(log_std)
    samples = get_uniform_samples(len(angles) * 8, mean, std * 2 * np.pi)
    transformed_samples = samples + dist_net(samples)
    sorted_samples, _ = th.sort(transformed_samples, dim=0)

    diffs = sorted_samples.reshape(len(points), -1) - sorted_points.unsqueeze(1)
    loss = th.sqrt(th.mean(diffs * diffs))
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Loss: {:.2f}".format(loss.item()))
        fig = plt.figure(figsize=(8,4))

        seaborn.distplot(var_to_np(sorted_points), color=seaborn.color_palette()[0], bins=80, rug=True)
        seaborn.distplot(var_to_np(sorted_samples), color=seaborn.color_palette()[1], bins=80, rug=True)
        display(fig)
        plt.close(fig)
        orig_sorted_samples = th.sort(samples, dim=0)[0]
        transformed_samples = orig_sorted_samples + dist_net(orig_sorted_samples)
        fig = plt.figure(figsize=(8,4))
        plt.plot(var_to_np(orig_sorted_samples), var_to_np(transformed_samples))
        display(fig)
        plt.close(fig)

### DSF in module

In [ ]:
def inv_sigmoid(p, eps=1e-8):
    return th.log(p + eps) - th.log(1-p + eps)

class DSFTransform(nn.Module):
    def __init__(self, n_units):
        super(DSFTransform, self).__init__()
        self.weights = nn.Parameter(th.randn(200) * 0.01, requires_grad=True)
        self.bias = nn.Parameter(th.randn(200) * 0.01, requires_grad=True)
        # make close to saturation of sigmoid to make transformation close
        # to identity at start
        self.alphas = nn.Parameter(th.randn(200) * 0.01 + 4, requires_grad=True)
        
    def forward(self, x):
        return inv_sigmoid(th.mean(th.sigmoid(self.alphas.unsqueeze(0)) * th.sigmoid(x *
                                   th.exp(self.weights).unsqueeze(0) 
                                  + self.bias.unsqueeze(0)), dim=1))

In [ ]:
from disttransform import DSFTransform

In [ ]:
i_bp = 18
points = np_to_var(angles[:,i_bp], dtype=np.float32)
sorted_points = th.sort(points, dim=0)[0].detach()

set_random_seeds(13123, False)

dsf_transform = DSFTransform(200)

samples = th.linspace(-3,3).unsqueeze(1)
out = dsf_transform(samples)

# 0.43 without transformation
mean = th.zeros(1, requires_grad=True)
log_std = th.zeros(1, requires_grad=True)
optim = th.optim.Adam([mean,log_std,] + list(dsf_transform.parameters()), lr=1e-1)

n_epochs = 300
for i_epoch in range(n_epochs):
    std = th.exp(log_std)

    samples = get_uniform_samples(len(angles) * 8, mean, std * 2 * np.pi)
    transformed_samples = dsf_transform(samples)
    sorted_samples, _ = th.sort(transformed_samples, dim=0)

    diffs = sorted_samples.reshape(len(points), -1) - sorted_points.unsqueeze(1)
    loss = th.sqrt(th.mean(diffs * diffs))
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Loss: {:.2f}".format(loss.item()))
        fig = plt.figure(figsize=(8,4))

        seaborn.distplot(var_to_np(sorted_points), color=seaborn.color_palette()[0], bins=80, rug=True)
        seaborn.distplot(var_to_np(sorted_samples), color=seaborn.color_palette()[1], bins=80, rug=True)
        display(fig)
        plt.close(fig)
        orig_sorted_samples = th.sort(samples, dim=0)[0]
        transformed_samples = dsf_transform(orig_sorted_samples)
        fig = plt.figure(figsize=(8,4))
        plt.plot(var_to_np(orig_sorted_samples), var_to_np(transformed_samples))
        display(fig)
        plt.close(fig)